In [ ]:
#%%

from dotenv import load_dotenv
load_dotenv("../.env")

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent

In [ ]:
#%%

# 初始化大模型
llm = ChatOpenAI(model='gpt-4o', temperature=0.5)

In [ ]:
#%%

# 设置工具
tools = load_tools(["serpapi", "llm-math"], llm=llm)
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='9a385efcc26ec0f5eb7a576dedd111bb8d8a3e2b616eb78a302db84f1a435d75', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='9a385efcc26ec0f5eb7a576dedd111bb8d8a3e2b616eb78a302db84f1a435d75', aiosession=None)>),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a 

In [ ]:
#%%

# 设置提示模板
template = ('''
    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:\n\n'
    '{tools}\n\n
    Use the following format:\n\n'
    'Question: the input question you must answer\n'
    'Thought: you should always think about what to do\n'
    'Action: the action to take, should be one of [{tool_names}]\n'
    'Action Input: the input to the action\n'
    'Observation: the result of the action\n'
    '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
    'Thought: I now know the final answer\n'
    'Final Answer: the final answer to the original input question\n\n'
    'Begin!\n\n'
    'Question: {input}\n'
    'Thought:{agent_scratchpad}'
    '''
)

prompt = PromptTemplate.from_template(template)

In [ ]:
#%%

prompt.pretty_print()


    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:

'
    '{tools}


    Use the following format:

'
    'Question: the input question you must answer
'
    'Thought: you should always think about what to do
'
    'Action: the action to take, should be one of [{tool_names}]
'
    'Action Input: the input to the action
'
    'Observation: the result of the action
'
    '... (this Thought/Action/Action Input/Observation can repeat N times)
'
    'Thought: I now know the final answer
'
    'Final Answer: the final answer to the original input question

'
    'Begin!

'
    'Question: {input}
'
    'Thought:{agent_scratchpad}'
    


In [ ]:
#%%

# 初始化Agent
agent = create_react_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.', 'tool_names': 'Search, Calculator'}, template="\n    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:\n\n'\n    '{tools}\n\n\n    Use the following format:\n\n'\n    'Question: the input question you must answer\n'\n    'Thought: you should always think about what to do\n'\n    'Action: the action to take, should be one 

In [ ]:
#%%

# 构建AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               handle_parsing_errors=True,
                               verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.', 'tool_names': 'Search, Calculator'}, template="\n    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:\n\n'\n    '{tools}\n\n\n    Use the following format:\n\n'\n    'Question: the input question you must answer\n'\n    'Thought: you should always think about what 

In [ ]:
#%%

# 执行AgentExecutor
agent_executor.invoke({"input":
                       """目前市场上玫瑰花的一般进货价格是多少？\n
                       如果我在此基础上加价5%，应该如何定价？"""})



> Entering new AgentExecutor chain...
'Question: 目前市场上玫瑰花的一般进货价格是多少？如果我在此基础上加价5%，应该如何定价？

Thought: 我需要先找到目前市场上玫瑰花的一般进货价格，然后计算加价5%后的定价。

Action: Search
Action Input: 目前市场上玫瑰花的一般进货价格
['一朵普通红玫瑰的价格大约在2-5元之间。 2、一般鲜花店标准售价一般而言，在城市中心或者商业区内的鲜花店出售的玫瑰花价格相对较高，可能会在3-40元/支 ...', '元旦来临，云南鲜切花市场迎来销售高峰期，价格飙升，卡罗拉红玫瑰平时10-20元一把，现在能卖到40-50元。涨价潮可能持续到春节。 “昨天拍一支花可能 ...', '以玫瑰为例，一扎（20枝）产自云南的普通C级玫瑰花往年价格为20至30元，今年最高价已涨至200至300元不等，价格翻了10倍。寓意长长久久的999朵玫瑰花束售价 ...', '今年由于气温回升较快，玫瑰花已经开始采摘。平阴县玫瑰鲜花蕾开秤价格从2.8元/斤，迅速攀升至4月27日的3.5元/斤，开秤价格平均每斤比去年同期上涨1 ...', "在记者走访了解中，不少商户直呼“今年七夕节玫瑰花价格是近几年来最低的一次”。“红玫瑰一捆20支的价格一般在25-40元，像'弗洛伊德'这类特定花色的精品玫瑰 ...", '我公司每天会按照我们网站上公布的鲜花报价发货,如有我们网站上公布的价格与实际的发货价格不符，请与我们网站上的客服联系或者直接致电给我们询问. 鲜花等级说明：. A级 ...', '“今年的玫瑰进价尤其贵，相较去年基本上翻了3倍。”有批发市场店主说。记者走访上海的灵石路批发市场及嘉艺花卉市场发现，鲜花大多从云南昆明批发过来，“ ...', '周先生说，由于临近情人节，玫瑰花的销量在逐天上涨，价格也比平时翻了好几倍。“拿红玫瑰举例，平时这个品种一把进价在30元左右，一把20支，现在已经涨到 ...', '店家告诉记者，这是专门为今年兔年打造的情人节花束，共有11支玫瑰，此类包装精致的玫瑰花花束价格都在200元左右，包装费在50-60元左右。常见的用包装纸 ...', '除实体店，玫瑰花在昆明斗南鲜花批发市场线上交易也不便宜。 3月6日，“斗南花城”平台单头玫瑰“蜜桃雪山”价

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info  in LangSmith API. Please confirm your internet connection.. ProxyError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by ProxyError(\'Cannot connect to proxy.\', RemoteDisconnected(\'Remote end closed connection without response\')))"))')


目前市场上玫瑰花的进货价格因品种和质量不同而有所差异。根据搜索结果，普通红玫瑰的价格大约在2-5元/支，而一些特定品种的价格可能会更高。例如：

- 普通红玫瑰：2-5元/支
- 一扎（20枝）普通C级玫瑰：25-40元/扎

假设我们以普通红玫瑰的平均进货价格3.5元/支来计算，如果在此基础上加价5%，该如何定价？

Action: Calculator
Action Input: 3.5 * 1.05
Answer: 3.6750000000000003'Question: 目前市场上玫瑰花的一般进货价格是多少？如果我在此基础上加价5%，应该如何定价？

Thought: 我需要先找到目前市场上玫瑰花的一般进货价格，然后计算加价5%后的定价。

Action: Search
Action Input: 目前市场上玫瑰花的一般进货价格
['一朵普通红玫瑰的价格大约在2-5元之间。 2、一般鲜花店标准售价一般而言，在城市中心或者商业区内的鲜花店出售的玫瑰花价格相对较高，可能会在3-40元/支 ...', '元旦来临，云南鲜切花市场迎来销售高峰期，价格飙升，卡罗拉红玫瑰平时10-20元一把，现在能卖到40-50元。涨价潮可能持续到春节。 “昨天拍一支花可能 ...', '以玫瑰为例，一扎（20枝）产自云南的普通C级玫瑰花往年价格为20至30元，今年最高价已涨至200至300元不等，价格翻了10倍。寓意长长久久的999朵玫瑰花束售价 ...', '今年由于气温回升较快，玫瑰花已经开始采摘。平阴县玫瑰鲜花蕾开秤价格从2.8元/斤，迅速攀升至4月27日的3.5元/斤，开秤价格平均每斤比去年同期上涨1 ...', "在记者走访了解中，不少商户直呼“今年七夕节玫瑰花价格是近几年来最低的一次”。“红玫瑰一捆20支的价格一般在25-40元，像'弗洛伊德'这类特定花色的精品玫瑰 ...", '我公司每天会按照我们网站上公布的鲜花报价发货,如有我们网站上公布的价格与实际的发货价格不符，请与我们网站上的客服联系或者直接致电给我们询问. 鲜花等级说明：. A级 ...', '“今年的玫瑰进价尤其贵，相较去年基本上翻了3倍。”有批发市场店主说。记者走访上海的灵石路批发市场及嘉艺花卉市场发现，鲜花大多从云南昆明批发过来，“ ...', '周先生说，由于临近情人节，玫瑰花的销量在逐天上涨，价格也

{'input': '目前市场上玫瑰花的一般进货价格是多少？\n\n                       如果我在此基础上加价5%，应该如何定价？',
 'output': '目前市场上普通红玫瑰的平均进货价格大约为3.5元/支，如果在此基础上加价5%，定价应为3.675元/支。'}